In [1]:
import pandas as pd
import numpy as np
import requests
import re
import nltk
from nltk.stem import WordNetLemmatizer
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
wnl = WordNetLemmatizer()
KEY = '13A3E641-0C10-317C-9C08-7EF10ACA1519'
DOMAIN = 'http://quickstats.nass.usda.gov/api'
cats = ['sector_desc','group_desc','statisticcat_desc','state_alpha']
#Methods
GET = '/api_GET/?key='+ KEY
VALUES = '/get_param_values/?key='+ KEY
COUNTS = '/get_counts/?key='+ KEY

# Dep Ratio 

In [2]:
ratio = pd.read_excel('Dep_Ratio.xlsx',sheetname = 1)
#Hard code: 'legume seeds' to 'legume'. "Plum/pune" to "Plum","Honeydew" to "Honeydew melons",
ratio = ratio.rename(columns={'Crop':'crop',
              'Dependence on insect pollination (D)':'D',
              'Proportion of pollinators that are domesticated exotic bees (P)':'P',
              'Proportion of pollinators that are native bees (1 – P)':'1-P'})
ratio ['crop'] = ratio ['crop'].apply(lambda x: wnl.lemmatize(str(x).lower()))
ratio=ratio[ratio.D.notnull()]
ratio["DN"]=ratio["D"]*ratio["P"]
ratio["DH"]=ratio["D"]*ratio["1-P"]
ratio

,crop,D,P,1-P,DN,DH
1,almond,1.0,1.00,0.00,1.00,0.00
2,apple,1.0,0.90,0.10,0.90,0.10
3,apricot,0.7,0.80,0.20,0.56,0.14
4,avocado,1.0,0.90,0.10,0.90,0.10
5,blueberry,1.0,0.90,0.10,0.90,0.10
6,boysenberry,0.8,0.90,0.10,0.72,0.08
7,brambleberry,0.8,0.90,0.10,0.72,0.08
8,cherry,0.9,0.90,0.10,0.81,0.09
9,grapefruit,0.8,0.90,0.10,0.72,0.08
10,lemon,0.2,0.10,0.90,0.02,0.18


### Join Dep Ratio with Crop_code

In [3]:
crop_code = pd.read_excel('CDL_AtrributeTable_2016_byStats.xlsx',sheetname = 0)
crop_code = crop_code.rename(columns={' Category':'crop',
                                   'Value':'crop_code'})
crop_code=crop_code[["crop","crop_code"]]
crop_code['crop'] = crop_code['crop'].apply(lambda x: wnl.lemmatize(str(x).lower()))
ratio_with_code= pd.merge(ratio, crop_code, how='left', on='crop')
ratio_with_code=ratio_with_code.sort_values(by=['crop_code'])
ratio_with_code.to_excel("Dep_Ratio_with_CropCode.xlsx")
ratio_with_code

,crop,D,P,1-P,DN,DH,crop_code
41,cotton,0.2,0.80,0.20,0.16,0.04,2.0
45,soybean,0.1,0.50,0.50,0.05,0.05,5.0
47,sunflower,1.0,0.90,0.10,0.90,0.10,6.0
43,peanut,0.1,0.20,0.80,0.02,0.08,10.0
44,rape seed,1.0,0.90,0.10,0.90,0.10,34.0
40,alfalfa,1.0,0.95,0.05,0.95,0.05,36.0
46,sugarbeets,0.1,0.20,0.80,0.02,0.08,41.0
39,watermelon,0.7,0.90,0.10,0.63,0.07,48.0
35,onion,1.0,0.90,0.10,0.90,0.10,49.0
32,cucumber,0.9,0.90,0.10,0.81,0.09,50.0


# CDL Attribute Table

In [4]:
df = pd.read_excel('CDL_AtrributeTable_2016_byStats.xlsx',
                   usecols=["State"," Category","Value","Pixel_Count","Acreage"],
                   sheetname=None, ignore_index=True)
cdl = pd.concat(df.values(),ignore_index=True)
cdl.columns = ['state', 'crop_code','crop','pixel_count','acres']

### Join CDL Attribute with State_code

In [5]:
state_code = pd.read_excel('state_code_ref.xlsx')
state_code = state_code.rename(columns={'Name and status of region':'state',
                                        'USCG':'state_code'})
state_code=state_code[["state","state_code"]]
cdl = pd.merge(cdl, state_code, how='left', on='state')

### Join  CDL Attribute with Dependency Ratio data

In [6]:
cdl = pd.merge(cdl, ratio_with_code, how='right', on='crop_code')

# Buffer and planned buffer CDL Attribute Table

In [7]:
buffer = pd.read_csv('all_states_gis_buffer.csv',usecols=["State_code","VALUE","COUNT"])
buffer.columns = ['pixel_count', 'state_code','crop_code']
buffer['acres']=buffer['pixel_count']*0.222394
buffer = pd.merge(buffer, state_code, how='left', on='state_code')
buffer = pd.merge(buffer, ratio_with_code, how='right', on='crop_code')

In [8]:
planned = pd.read_csv('all_states_gis_buffer_planned.csv',usecols=["State_code","VALUE","COUNT"])
planned.columns = ['pixel_count', 'state_code','crop_code']
planned['acres']=planned ['pixel_count']*0.222394
planned = pd.merge(planned , state_code, how='left', on='state_code')
planned = pd.merge(planned , ratio_with_code, how='right', on='crop_code')

# NASS Data 

In [9]:
#helpfer function
commodity_list = list(pd.read_excel("CropName_Checklist.xlsx").crop_NASS)

def get_nass_data(category,year):    
    filename = 'NASS_survey_'+ category + str(year) +'.csv'

    f = open(filename,'wb')
    p_get = {'source_desc':'SURVEY',
     'sector_desc': 'CROPS',
     'group_desc': ["FIELD CROPS","FRUIT & TREE NUTS","VEGETABLES","HORICULTURE"],
     'commodity_desc': commodity_list,
     'statisticcat_desc':category,
     'year':[year],
     'reference_period_desc':['YEAR','MARKETING YEAR'],
     'freq_desc':"ANNUAL",
     'agg_level_desc':"STATE",
     'format':'CSV'}
    data = requests.get(DOMAIN+GET, params=p_get).content
    f.write(data) 
    f.close()
    print('Exported '+filename+'....')
    
        
def ref_feature(row):
    f = row['short_desc'].split("-")[:-1]
    f.append(row['state_alpha'])
    string = '-'.join(str(v) for v in f)
    return string

def clean_nass_data(filename):
    df = pd.read_csv(filename)
    df['ref_feature'] = df.apply (lambda row: ref_feature(row),axis=1)
    df =df[['commodity_desc','short_desc','class_desc','state_ansi','year','unit_desc','Value','ref_feature']]
    df['Value'] = df['Value'].apply(lambda x: float(str(x).replace(',','')))
    df['Value'] = df['Value'].astype(float)
    df = df.dropna()
    return df

### Getting NASS Data -  PRICE RECEIVED, YIELD, AREA HARVESTED

In [731]:
get_nass_data('PRICE RECEIVED',2016)
get_nass_data('YIELD',2016)
get_nass_data('AREA HARVESTED',2016)
get_nass_data('PRODUCTION',2016)

Exported NASS_survey_PRICE RECEIVED2016.csv....
Exported NASS_survey_YIELD2016.csv....
Exported NASS_survey_AREA HARVESTED2016.csv....
Exported NASS_survey_PRODUCTION2016.csv....


#### Please manually remove non numerical value 

In [10]:
Price = clean_nass_data('NASS_survey_PRICE RECEIVED2016.csv')
Yield = clean_nass_data('NASS_survey_YIELD2016.csv')
Area= clean_nass_data('NASS_survey_AREA HARVESTED2016.csv')
Production= clean_nass_data('NASS_survey_PRODUCTION2016.csv')
Production = Production[Production.unit_desc=="$"]

In [11]:
s_p = set(Price.commodity_desc.unique())
s_y = set(Yield.commodity_desc.unique())
s_a = set(Area.commodity_desc.unique())
s_prod = set(Production.commodity_desc.unique())

### Join NASS Data

##### 1)when crop have price data for "total price" and "yield". per acre = yield*price

In [12]:
Price1=Price[Price.class_desc=="ALL CLASSES"]
Yield1=Yield[Yield.class_desc=="ALL CLASSES"]

r1_i = pd.merge(Price1,Yield1,on='ref_feature',how='inner')
r1_i['Price/Acre'] = r1_i.Value_x*r1_i.Value_y
r1_i = r1_i[['year_x','state_ansi_x','commodity_desc_x','Price/Acre']]
r1_i.columns = ['year','state_ansi','crop','Price/Acre']

#Check for crops that need manual treatment 
r1_o = pd.merge(Price1,Yield1,on='ref_feature',how='outer')
s1 = set(r1_i.crop.unique())
s2 = set(r1_o[r1_o.isnull().any(axis=1)].commodity_desc_x.unique())
print(len(s1))
print(s2-s1)

export = r1_o[r1_o.commodity_desc_x.isin(['CRANBERRIES', 'ALMONDS', 'BOYSENBERRIES', 'RASPBERRIES']) |
             r1_o.commodity_desc_y.isin(['CRANBERRIES', 'ALMONDS', 'BOYSENBERRIES', 'RASPBERRIES'])]
export.to_csv('r1_tobecleaned.csv')

25
{nan, 'ALMONDS', 'CRANBERRIES', 'RASPBERRIES', 'BOYSENBERRIES'}


In [13]:
#Clean the data manually
r1_i = r1_i.append(pd.read_csv('r1_cleaned.csv'), ignore_index=True)
r1_i

,Price/Acre,crop,state_ansi,year
0,231.000,HAY,1.0,2016
1,459.000,HAY,2.0,2016
2,1251.540,HAY,4.0,2016
3,204.000,HAY,5.0,2016
4,829.930,HAY,6.0,2016
5,391.090,HAY,8.0,2016
6,308.800,HAY,9.0,2016
7,517.140,HAY,10.0,2016
8,432.000,HAY,12.0,2016
9,230.000,HAY,13.0,2016


##### 2)when crop doesn't have price data for "total price", but price data for each sub-classification. per acre = (subclass_yield*subclass_price)weighted by total production value. 

In [14]:
Price2=Price[Price.class_desc!="ALL CLASSES"]
Yield2=Yield[Yield.class_desc!="ALL CLASSES"]

r2_i = Price2.merge(Yield2,on='ref_feature',how='inner').merge(Production,on='ref_feature',how='inner')
r2_i['subclass_Price/Acre'] = r2_i.Value_x*r2_i.Value_y
grouped = r2_i.groupby(["year_x","state_ansi_x","commodity_desc_x"])
def wavg(group):
    d = group['subclass_Price/Acre']
    w = group['Value']
    return (d*w).sum()/w.sum()
r2_i = grouped.apply(wavg).reset_index()
r2_i.columns = ['year','state_ansi','crop','Price/Acre']
s3 = set(r2_i.crop.unique())
s3

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


{'COTTON', 'HAY', 'MELONS', 'ONIONS', 'SUNFLOWER'}

In [15]:
#Deal with melons - Watermelon and Honeydrew
r2_o = pd.merge(Price2,Yield2,on='ref_feature',how='outer')
export2 = r2_o[r2_o.commodity_desc_x.isin(['HAY','MELONS']) |
             r2_o.commodity_desc_y.isin(['HAY','MELONS'])]
export2.to_csv('r2_tobecleaned.csv')
r2_i = r2_i.append(pd.read_csv('r2_cleaned.csv'), ignore_index=True)
r2_i

,Price/Acre,crop,state_ansi,year
0,701.480000,COTTON,1.0,2016
1,231.000000,HAY,1.0,2016
2,1015.650000,COTTON,4.0,2016
3,1269.852814,HAY,4.0,2016
4,4844.262814,MELONS,4.0,2016
5,736.375000,COTTON,5.0,2016
6,208.524017,HAY,5.0,2016
7,956.391127,HAY,6.0,2016
8,6886.489472,MELONS,6.0,2016
9,4145.550000,ONIONS,6.0,2016


##### 3)when crop doesn't have price data at all, but have production and area data. we use another method to calcualte price per acre: price per acre = total production in value/total harvest area

In [16]:
s4 = set(r1_i.crop.unique())
s5 = set(r2_i.crop.unique())
print(s_prod & s_a - (s4|s5))

{'BLUEBERRIES'}


In [17]:
Production1 = Production[Production.commodity_desc.isin(['BLUEBERRIES'])]
Area1 = Area[Area.commodity_desc.isin(['BLUEBERRIES'])]
r3_o=pd.merge(Production1,Area1,on='ref_feature',how='outer')
r3_o.to_csv('r3_tobecleaned.csv')
r3 = pd.read_csv('r3_cleaned.csv')
r3

,crop,state_ansi,year,Price/Acre
0,BLUEBERRIES,6,2016,16994.531250
1,BLUEBERRIES,12,2016,11416.170210
2,BLUEBERRIES,13,2016,5225.147929
3,BLUEBERRIES,26,2016,6421.182266
4,BLUEBERRIES,28,2016,6081.666667
5,BLUEBERRIES,34,2016,6386.021505
6,BLUEBERRIES,36,2016,3594.444444
7,BLUEBERRIES,37,2016,9295.000000
8,BLUEBERRIES,41,2016,8788.235294
9,BLUEBERRIES,53,2016,7015.522388


In [18]:
nass_all = r1_i.append(r2_i,ignore_index=True).append(r3,ignore_index=True)
nass_all = nass_all.groupby(["year","state_ansi","crop"]).max().reset_index()
nass_df = nass_all[~nass_all.crop.isin(['HAY','MELONS'])]

### Add state code and crop code

In [19]:
crop_final = pd.read_excel("CropName_Checklist.xlsx")
nass_df_final = pd.merge(nass_df, crop_final, how='outer', on='crop')[['crop','state_ansi','year','crop_code_CDL','Price/Acre']]
nass_df_final = nass_df_final.rename(columns={'state_ansi':'state_code','crop_code_CDL':'crop_code'})
nass_df_final

,crop,state_code,year,crop_code,Price/Acre
0,COTTON,1.0,2016.0,2.0,701.480000
1,COTTON,4.0,2016.0,2.0,1015.650000
2,COTTON,5.0,2016.0,2.0,736.375000
3,COTTON,12.0,2016.0,2.0,625.116000
4,COTTON,13.0,2016.0,2.0,646.560000
5,COTTON,20.0,2016.0,2.0,758.310000
6,COTTON,22.0,2016.0,2.0,655.422000
7,COTTON,28.0,2016.0,2.0,855.763000
8,COTTON,29.0,2016.0,2.0,705.511000
9,COTTON,35.0,2016.0,2.0,636.540000


# Join NASS data to CDL Attribute Table

In [20]:
final_df = cdl.merge(nass_df_final,on=['state_code','crop_code'],how='right')
final_df = final_df[final_df.state!='National']
final_df = final_df[np.isfinite(final_df['Price/Acre'])]
final_df["state"] = final_df["state"].apply(lambda x: str(x).replace(' ',''))

In [21]:
final_df['Value_P']=final_df['Price/Acre']*final_df['acres']*final_df['P']
final_df['Value_NP']=final_df['Price/Acre']*final_df['acres']*final_df['DN']
final_df['Value_HP']=final_df['Price/Acre']*final_df['acres']*final_df['DH']
df_byState= final_df.groupby(["state"]).sum()
df_byCrop= final_df.groupby(["crop_x"]).sum()

writer = pd.ExcelWriter('output_2016.xlsx')
final_df.to_excel(writer,'Summary')
df_byState.to_excel(writer,'byState')
df_byCrop.to_excel(writer,'byCrop')
writer.save()

# Join NASS data to Buffer and Planned Buffer CDL Attribute Table

In [24]:
final_df_buffer = buffer.merge(nass_df_final,on=['state_code','crop_code'],how='right')
final_df_buffer = final_df_buffer[final_df_buffer.state!='National']
final_df_buffer = final_df_buffer[np.isfinite(final_df_buffer['Price/Acre'])]
final_df_buffer["state"] = final_df_buffer["state"].apply(lambda x: str(x).replace(' ',''))
final_df_buffer['Value_P']=final_df_buffer['Price/Acre']*final_df_buffer['acres']*final_df_buffer['P']
final_df_buffer['Value_NP']=final_df_buffer['Price/Acre']*final_df_buffer['acres']*final_df_buffer['DN']
final_df_buffer['Value_HP']=final_df_buffer['Price/Acre']*final_df_buffer['acres']*final_df_buffer['DH']
df_byState_buffer= final_df_buffer.groupby(["state"]).sum()
df_byCrop_buffer= final_df_buffer.groupby(["crop_x"]).sum()

final_df_planned = planned.merge(nass_df_final,on=['state_code','crop_code'],how='right')
final_df_planned = final_df_planned[final_df_planned.state!='National']
final_df_planned = final_df_planned[np.isfinite(final_df_planned['Price/Acre'])]
final_df_planned["state"] = final_df_planned["state"].apply(lambda x: str(x).replace(' ',''))
final_df_planned['Value_P']=final_df_planned['Price/Acre']*final_df_planned['acres']*final_df_planned['P']
final_df_planned['Value_NP']=final_df_planned['Price/Acre']*final_df_planned['acres']*final_df_planned['DN']
final_df_planned['Value_HP']=final_df_planned['Price/Acre']*final_df_planned['acres']*final_df_planned['DH']
df_byState_planned= final_df_planned.groupby(["state"]).sum()
df_byCrop_planned= final_df_planned.groupby(["crop_x"]).sum()

writer2 = pd.ExcelWriter('output_buffer_2016.xlsx')
final_df_buffer.to_excel(writer2,'Summary_buffer')
df_byState_buffer.to_excel(writer2,'byState_buffer')
df_byCrop_buffer.to_excel(writer2,'byCrop_buffer')
final_df_planned.to_excel(writer2,'Summary_planned')
df_byState_planned.to_excel(writer2,'byState_planned')
df_byCrop_planned.to_excel(writer2,'byCrop_planned')
writer2.save()